In [7]:
# Notebook til at se på SSI's data for antigentests
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "Dark2"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams['lines.markersize'] = 10
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]


saveFigures is set to: True
Done loading packages


In [8]:
print('Make sure to run "get_data" first, so the most recent data is used')

Make sure to run "get_data" first, so the most recent data is used


In [9]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])

print(latestdir)
print(latestDate)

d:\Pandemix\Github\DanskeData/ssi_data\SSI_data_2021-04-14
2021-04-14 00:00:00


In [10]:
# Load the data
fulldfPCR = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
fulldf = pd.read_csv(latestdir+'/Test_pos_over_time_antigen.csv',delimiter = ';',dtype=str)

# Cut out the last two rows of summary numbers
dfPCR = fulldfPCR.iloc[:-2,:].copy()
df = fulldf.iloc[:-2,:].copy()

In [11]:

dfPCR["PosPct"] = pd.to_numeric(dfPCR["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfPCR["Date"] = dfPCR["Date"].astype('datetime64[D]')
df["PosPct"] = pd.to_numeric(df["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
df["Date"] = df["Date"].astype('datetime64[D]')

rows_to_fix_period_in = ["NewPositive","NotPrevPos","PrevPos","Tested","Tested_kumulativ"]
for name in rows_to_fix_period_in:
    df[name] = pd.to_numeric(df[name].astype(str).apply(lambda x: x.replace('.','')))
    dfPCR[name] = pd.to_numeric(dfPCR[name].astype(str).apply(lambda x: x.replace('.','')))


In [12]:
# Plot different things
firstDate = np.datetime64('2020-03-01') # All tests
firstDate = np.datetime64('2021-02-01') # Antigen test
today = date.today()

# fig,ax1 = plt.subplots(1,1)
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.plot(df.Date,df.NewPositive,label='Nye positive')
# ax1.plot(df.Date,df.Tested)

ax2.plot(df.Date,100*np.divide(df.NewPositive,df.Tested),label='Positiv procent')
ax2.plot(df.Date,0.1*df.PosPct,label='Positiv procent (Afrundet)')

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)

ax1.set_xlim([firstDate,today])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Figure to compare antigen positive procent with PCR

In [13]:
# Plot different things
firstDate = np.datetime64('2020-03-01') # All tests
firstDate = np.datetime64('2021-02-01') # Antigen test
today = date.today()

# fig,ax1 = plt.subplots(1,1)
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)

# ax1.plot(df.Date,df.NewPositive,label='Nye positive')
ax1.plot(df.Date,df.Tested,label='Tested, antigen')
ax1.plot(dfPCR.Date,dfPCR.Tested,label='Tested, PCR')

ax2.plot(df.Date,df.NewPositive,label='New positive, antigen')
ax2.plot(dfPCR.Date,dfPCR.NewPositive,label='New positive, PCR')

posPctAG = 100*np.divide(df.NewPositive,df.Tested)
posPctPCR = 100*np.divide(dfPCR.NewPositive,dfPCR.Tested)
ax3.plot(df.Date,posPctAG,label='Positiv procent, antigen')
ax3.plot(dfPCR.Date,posPctPCR,label='Positiv procent, PCR')
# ax2.plot(df.Date,df.PosPct,label='Positiv procent (Afrundyet)')
# ax2.plot(dfPCR.Date,0.1*dfPCR.PosPct,label='Positiv procet (Afrundet)')

backToShow = -30

ax1.set_ylim(bottom=0)
ax2Top = np.max(dfPCR.NewPositive[backToShow:])*1.1
ax2.set_ylim(bottom=0,top=ax2Top)
ax3Top =np.max(posPctPCR[backToShow:])*1.1
ax3.set_ylim(bottom=0,top=ax3Top)

ax1.legend()
ax2.legend()
ax3.legend()

ax1.grid()
ax2.grid()
ax3.grid()

# ax1.set_xlim([firstDate,today])
lastDate = df.iloc[-1].Date
ax1.set_xlim([firstDate,lastDate])
# ax1.set_xlim([firstDate,today-np.timedelta64(2,'D')])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b\n%Y'))

plt.tight_layout()

if saveFigures:
    fig.savefig('figs/PosPctComparison')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# df.NewPositive

# Figures of the "Antigentest_pr_dag.csv" file

In [29]:
# Load the data
df2 = pd.read_csv(latestdir+'/Antigentests_pr_dag.csv',delimiter = ';')

# # Remove the first two rows which are from april 2020
# df2 = df2.iloc[2:,:].copy()

In [32]:
df2["Dato"] = df2["Dato"].astype('datetime64[D]')
df2["Andel_AGpos_PCRkonf_pct"] = pd.to_numeric(df2["Andel_AGpos_PCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))
df2["Andel_AGminusPCRkonf_pct"] = pd.to_numeric(df2["Andel_AGminusPCRkonf_pct"].astype(str).apply(lambda x: x.replace(',','.')))

In [33]:
df2

,Dato,AG_testede,AG_pos,AGpos_PCRkonf,Andel_AGpos_PCRkonf_pct,AGpos_minusPCRkonf,Andel_AGminusPCRkonf_pct,AGpos_PCRpos,AGposPCRneg,AGnegPCRpos,AGnegPCRneg
0,2021-02-01,4764,8,4,50.000000,4,50.000000,2,2,2,293
1,2021-02-02,4592,8,5,62.500000,3,37.500000,5,0,5,308
2,2021-02-03,4813,9,5,55.555556,4,44.444444,5,0,6,343
3,2021-02-04,4453,4,2,50.000000,2,50.000000,2,0,5,322
4,2021-02-05,4319,7,5,71.428571,2,28.571429,4,1,6,280
5,2021-02-06,2572,8,6,75.000000,2,25.000000,6,0,3,262
6,2021-02-07,2994,11,7,63.636364,4,36.363636,7,0,6,326
7,2021-02-08,6876,11,6,54.545455,5,45.454545,6,0,5,439
8,2021-02-09,6014,6,4,66.666667,2,33.333333,4,0,5,390
9,2021-02-10,6999,13,5,38.461538,8,61.538462,4,1,1,498


In [17]:
# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,sharex=True)

ax1.plot(df2.Dato,df2.AG_testede,label='AG tests')
ax1.plot(dfPCR.Date,dfPCR.Tested,label='PCR tests')
# ax1.plot(df2.Dato,df2.AG_pos)
# ax1.plot(df2.Dato,100*np.divide(df2.AG_pos,df2.AG_testede),label='Andel af positive AG tests')

# firstToShow = -75
# ax1.plot(df.Date,posPctAG,label='Positiv procent, antigen')
# ax1.plot(dfPCR.Date[firstToShow:],posPctPCR[firstToShow:],label='Positiv procent, PCR')


unconf = df2['AGpos_minusPCRkonf']
unconfAndPos = unconf+df2['AGpos_PCRpos']
allAGPos = unconfAndPos+df2['AGposPCRneg']

# ax2.plot(df2.Dato,df2['AGpos_minusPCRkonf'])
# ax2.plot(df2.Dato,df2['AGpos_PCRpos'])
# ax2.plot(df2.Dato,df2['AGposPCRneg'])

# ax2.plot(df2.Dato,unconf)
# ax2.plot(df2.Dato,unconfAndPos)
# ax2.plot(df2.Dato,allAGPos)
# ax2.plot(df2.Dato,df2.AG_pos)

# ax2.plot(df2.Dato,np.divide(unconf,allAGPos),label='AG positiv, endnu ikke PCR testet')
# ax2.plot(df2.Dato,np.divide(unconfAndPos,allAGPos),label='AG positiv, PCR positiv')
# ax2.plot(df2.Dato,np.divide(allAGPos,allAGPos),label='AG positiv, PCR negativ')

ax2.fill_between(df2.Dato,100 * np.ones(df2.Dato.shape),label='AG positiv, PCR negativ')
ax2.fill_between(df2.Dato,100 * np.divide(unconfAndPos,allAGPos),label='AG positiv, PCR positiv')
ax2.fill_between(df2.Dato,100 * np.divide(unconf,allAGPos),color='gray',label='AG positiv, endnu ikke PCR testet')

allConf = df2.AGpos_PCRpos + df2.AGposPCRneg
ax3.plot(df2.Dato,100 * np.divide(df2.AGposPCRneg,allConf),label='AG positiv, PCR negativ')
# ax3.plot(df2.Dato,100 * np.divide(df2.AGpos_PCRpos,allConf),label='AG positiv, PCR positiv')


# sumNeg = np.sum(df2.AGnegPCRpos.values,df2.AGnegPCRneg.values)
sumNeg = df2.AGnegPCRneg.values + df2.AGnegPCRpos.values
# ax3.fill_between(df2.Dato,100 * np.ones(df2.Dato.shape),label='AG negativ, PCR positiv')
# ax3.fill_between(df2.Dato,100 * np.divide(df2.AGnegPCRneg,sumNeg),label='AG negativ, PCR negativ')
# ax3.fill_between(df2.Dato,df2.AGnegPCRneg)


# ax3.fill_between(df2.Dato,100 * np.ones(df2.Dato.shape),label='AG negativ, PCR negativ')
# ax3.fill_between(df2.Dato,100 * np.divide(df2.AGnegPCRpos,sumNeg),label='AG negativ, PCR positiv')
ax4.plot(df2.Dato,100 * np.divide(df2.AGnegPCRpos,sumNeg),label='AG negativ, PCR positiv')

meanWidth = 7
rmDate = rnTime(df2.Dato,meanWidth)
rmNegPos = rnMean(np.divide(df2.AGnegPCRpos,sumNeg),meanWidth)
ax4.plot(rmDate,100 * rmNegPos,'k:',label='7 dages løbende gennemsnit')


# ax1.legend(loc='upper left')
# ax2.legend(loc='upper left')
ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax4.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.set_ylim(bottom=0)
ax2.set_ylim([0,100])
ax3.set_ylim([0,100])
ax4.set_ylim([0,100])
# ax4.set_ylim([95,100])
ax4.set_ylim([0,3])

ax1.set_ylabel('Antal tests')
ax2.set_ylabel('Andel af tests [%]')
ax3.set_ylabel('Andel af PCR-\nkonfirmede tests [%]')
ax4.set_ylabel('Andel af tests [%]')
# ax1.set_xlim([df2.iloc[0,0],df2.iloc[-1,0]])

ax1.grid()
# ax2.grid()
ax3.grid()
ax4.grid()

lastDate = df2.iloc[-1].Dato
# ax1.set_xlim([np.datetime64('2021-02-01'),today])
ax1.set_xlim([np.datetime64('2021-02-01'),lastDate])

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b\n%Y'))
plt.tight_layout()

if saveFigures:
    fig.savefig('figs/AGprDagOpsummering')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
# df2

In [40]:

# fig,ax1 = plt.subplots(1,1)
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

# ax1.plot(df2.Dato,df2['AGnegPCRpos'],label='AG neg, PCR pos')
# ax1.plot(df2.Dato,df2['AGnegPCRneg'],label='AG neg, PCR neg')
# ax1.plot(df2.Dato,1e5*np.divide(df2['AGnegPCRpos'],df2['AG_testede']),label='AG neg, PCR pos')
# ax1.plot(df2.Dato,1e5*np.divide(df2['AGnegPCRneg'],df2['AG_testede']),label='AG neg, PCR neg')

# ax1.fill_between(df2.Dato,100 * np.ones(df2.Dato.shape),label='AG positiv, PCR negativ')
# ax1.fill_between(df2.Dato,100 * np.divide(unconfAndPos,allAGPos),label='AG positiv, PCR positiv')
# ax1.fill_between(df2.Dato,100 * np.divide(unconf,allAGPos),color='gray',label='AG positiv, endnu ikke PCR testet')

allConf = df2.AGpos_PCRpos + df2.AGposPCRneg
# ax1.plot(df2.Dato,100 * np.divide(df2.AGpos_PCRpos,allConf),label='AG positiv, PCR positiv')
ax1.fill_between(df2.Dato,100 * np.ones(df2.Dato.shape),color='xkcd:beige',label='AG positiv, PCR negativ')
ax1.fill_between(df2.Dato,100 * np.divide(df2.AGpos_PCRpos,allConf),color='xkcd:sky blue',label='AG positiv, PCR positiv')



totAGneg = df2['AGnegPCRpos'] + df2['AGnegPCRneg']
# ax2.plot(df2.Dato,np.divide(df2['AGnegPCRpos'],totAGneg),label='AG neg, PCR pos')
# ax2.plot(df2.Dato,100 * np.divide(df2['AGnegPCRneg'],totAGneg),label='AG negativ, PCR negativ')
ax2.fill_between(df2.Dato,100 * np.ones(df2.Dato.shape),color='xkcd:beige',label='AG negativ, PCR positive')
ax2.fill_between(df2.Dato,100 * np.divide(df2['AGnegPCRneg'],totAGneg),color='xkcd:sky blue',label='AG negativ, PCR negativ')

## Running means
meanWidth = 7
rmDate = rnTime(df2.Dato,meanWidth)
curPP = np.divide(df2.AGpos_PCRpos,allConf)
curNN = np.divide(df2['AGnegPCRneg'],totAGneg)
rmPP = rnMean(curPP,meanWidth)
rmNN = rnMean(curNN,meanWidth)

ax1.plot(rmDate,100 * rmPP,'k:',label=str(meanWidth) + ' dages gennemsnit')
ax2.plot(rmDate,100 * rmNN,'k:',label=str(meanWidth) + ' dages gennemsnit')

firstDateToUse = np.datetime64('2021-02-15')
indexToUse = (df2.Dato > firstDateToUse)
meanPP = np.mean(curPP[indexToUse])
meanNN = np.mean(curNN[indexToUse])
ax1.plot(df2.Dato[indexToUse],100 * np.ones(df2.Dato[indexToUse].shape) * meanPP,'g--',label='Gennemsnit fra '+str(firstDateToUse)+ ' og frem')
ax2.plot(df2.Dato[indexToUse],100 * np.ones(df2.Dato[indexToUse].shape) * meanNN,'g--',label='Gennemsnit fra '+str(firstDateToUse)+ ' og frem')


ax1.set_ylim([0,100])
ax2.set_ylim([95,100])

ax1.legend()
ax2.legend()
# ax1.grid()
# ax2.grid()

lastDate = df2.iloc[-1].Dato
# ax1.set_xlim([np.datetime64('2021-02-01'),today])
ax1.set_xlim([np.datetime64('2021-02-01'),lastDate])

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b\n%Y'))
plt.tight_layout()

if saveFigures:
    fig.savefig('figs/PCRverificeredeAGtest')

ax1.plot(df2.Dato,df2['Andel_AGpos_PCRkonf_pct'],color='m',label='Andel_AGpos_PCRkonf_pct')
ax1.legend()
if saveFigures:
    fig.savefig('figs/PCRverificeredeAGtest_Andel_AGpos_PCRkonf_pct')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [148]:


curdf = df2[df2.Dato <= np.datetime64('2021-03-23')].copy()

totPCR = curdf.AGpos_PCRpos+curdf.AGposPCRneg+curdf.AGnegPCRpos+curdf.AGnegPCRneg

# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
# ax1.plot(curdf.Dato,curdf.AGpos_PCRpos,label='AGpos_PCRpos')
# ax1.plot(curdf.Dato,curdf.AGposPCRneg,label='AGposPCRneg')
# ax1.plot(curdf.Dato,curdf.AGnegPCRpos,label='AGnegPCRpos')

# # ax1.plot(curdf.Dato,curdf.AGnegPCRneg,label='AGnegPCRneg')
# # ax1.plot(curdf.Dato,totPCR,label='Total')

# ax2.plot(curdf.Dato,100 * np.divide(curdf.AGpos_PCRpos,totPCR),label='AGpos_PCRpos')
# ax2.plot(curdf.Dato,100 * np.divide(curdf.AGposPCRneg,totPCR),label='AGposPCRneg')
# ax2.plot(curdf.Dato,100 * np.divide(curdf.AGnegPCRpos,totPCR),label='AGnegPCRpos')
# # ax2.plot(curdf.Dato,100 * np.divide(curdf.AGnegPCRneg,totPCR),label='AGnegPCRneg')
# # ax2.plot(curdf.Dato,100 * np.ones(curdf.Dato.shape),label='Total')

# ax1.legend()
# ax2.legend()

# ax1.set_ylabel('Antal')
# ax2.set_ylabel('Andel [%]')

# plt.tight_layout()




fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

totPCR = curdf.AGpos_PCRpos+curdf.AGposPCRneg+curdf.AGnegPCRpos+curdf.AGnegPCRneg
totPos = curdf.AGpos_PCRpos+curdf.AGposPCRneg
totNeg = curdf.AGnegPCRpos+curdf.AGnegPCRneg


# ax1.plot(curdf.Dato,curdf.AGnegPCRneg,label='AGnegPCRneg')
# ax1.plot(curdf.Dato,totPCR,label='Total')

ax1.plot(curdf.Dato,100 * np.divide(curdf.AGpos_PCRpos,totPos),label='AGpos_PCRpos')
ax1.plot(curdf.Dato,100 * np.divide(curdf.AGposPCRneg,totPos),label='AGposPCRneg')
ax2.plot(curdf.Dato,100 * np.divide(curdf.AGnegPCRpos,totNeg),label='AGnegPCRpos')
ax2.plot(curdf.Dato,100 * np.divide(curdf.AGnegPCRneg,totNeg),label='AGnegPCRneg')
# ax2.plot(curdf.Dato,100 * np.ones(curdf.Dato.shape),label='Total')



ax1.legend()
ax2.legend()

# ax1.set_ylabel('Antal')
# ax2.set_ylabel('Andel [%]')

plt.tight_layout()

# print(100*np.mean(np.divide(curdf.AGpos_PCRpos,totPos)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [178]:
curdf = df2[(df2.Dato <= np.datetime64('2021-03-23')) & (df2.Dato >= np.datetime64('2021-02-01'))].copy()
# curdf = df2[(df2.Dato <= np.datetime64('2021-03-23')) & (df2.Dato >= np.datetime64('2021-02-20'))].copy()
# curdf = df2[(df2.Dato <= np.datetime64('2021-03-21')) & (df2.Dato >= np.datetime64('2021-02-01'))].copy()

print('      | PCR +  | PCR -   |')
print('______|________|_________|')
print(f'AG +  | {curdf.AGpos_PCRpos.sum()}   | {curdf.AGposPCRneg.sum()}    | {curdf.AGpos_PCRpos.sum()+curdf.AGposPCRneg.sum()}')
print(f'AG -  | {curdf.AGnegPCRpos.sum()}   | {curdf.AGnegPCRneg.sum()}  | {curdf.AGnegPCRpos.sum()+curdf.AGnegPCRneg.sum()} ')
print(f'      | {curdf.AGpos_PCRpos.sum()+curdf.AGnegPCRpos.sum()}   | {curdf.AGposPCRneg.sum()+curdf.AGnegPCRneg.sum()}')
# curdf.AGnegPCRpos.sum() + curdf.AGpos_PCRpos.sum()
print('')
print(f'Total: {totPCR.sum()}')

ag_P_pcr_P = curdf.AGpos_PCRpos.sum()
ag_N_pcr_P = curdf.AGnegPCRpos.sum()
ag_P_pcr_N = curdf.AGposPCRneg.sum()
ag_N_pcr_N = curdf.AGnegPCRneg.sum()


# falskPos = 100*ag_P_pcr_N/(ag_P_pcr_P + ag_P_pcr_N)
# sandPos  = 100*ag_P_pcr_P/(ag_P_pcr_P + ag_P_pcr_N)
# falskNeg = 100*ag_N_pcr_P/(ag_N_pcr_P + ag_N_pcr_N)
# sandNeg  = 100*ag_N_pcr_N/(ag_N_pcr_P + ag_N_pcr_N)
# # falskPos = curdf.AGpos_PCRpos.sum()/(curdf.AGpos_PCRpos.sum()+curdf.AGposPCRneg.sum())
# # falskPos = curdf.AGpos_PCRpos.sum()/(curdf.AGpos_PCRpos.sum()+curdf.AGposPCRneg.sum())
# # falskPos = curdf.AGpos_PCRpos.sum()/(curdf.AGpos_PCRpos.sum()+curdf.AGposPCRneg.sum())
# # falskPos = curdf.AGpos_PCRpos.sum()/(curdf.AGpos_PCRpos.sum()+curdf.AGposPCRneg.sum())

# print(f'Falsk positiv (AG+,PCR-): {falskPos:2.2f} %')
# print(f' Sand positiv (AG+,PCR+): {sandPos:2.2f} %')
# print(f'Falsk negativ (AG-,PCR+): {falskNeg:2.2f} %')
# print(f' Sand negativ (AG-,PCR-): {sandNeg:2.2f} %')


falskPos = 100*ag_P_pcr_N/(ag_P_pcr_N + ag_N_pcr_N)
sandPos  = 100*ag_P_pcr_P/(ag_P_pcr_P + ag_N_pcr_P)
falskNeg = 100*ag_N_pcr_P/(ag_N_pcr_P + ag_P_pcr_P)
sandNeg  = 100*ag_N_pcr_N/(ag_N_pcr_N + ag_P_pcr_N)
# falskPos = curdf.AGpos_PCRpos.sum()/(curdf.AGpos_PCRpos.sum()+curdf.AGposPCRneg.sum())
# falskPos = curdf.AGpos_PCRpos.sum()/(curdf.AGpos_PCRpos.sum()+curdf.AGposPCRneg.sum())
# falskPos = curdf.AGpos_PCRpos.sum()/(curdf.AGpos_PCRpos.sum()+curdf.AGposPCRneg.sum())
# falskPos = curdf.AGpos_PCRpos.sum()/(curdf.AGpos_PCRpos.sum()+curdf.AGposPCRneg.sum())

print(f' Sand positiv (AG+,PCR+): {sandPos:2.2f} % af alle PCR+')
print(f'Falsk negativ (AG-,PCR+): {falskNeg:2.2f} % af alle PCR+')
# print(f' Sand negativ (AG-,PCR-): {sandNeg:2.2f} % af alle PCR-')
# print(f'Falsk positiv (AG+,PCR-): {falskPos:2.2f} % af alle PCR-')
sandAgP = 100*ag_P_pcr_P/(ag_P_pcr_N + ag_P_pcr_P)
# falskAgP = 100*ag_N_pcr_P/(ag_P_pcr_N + ag_P_pcr_P)
print(f' Sand positiv AG (AG+,PCR+): {sandAgP:2.2f} % af alle AG+')
print(f'Falsk positiv AG (AG+,PCR-): {100-sandAgP:2.2f} % af alle AG+')

      | PCR +  | PCR -   |
______|________|_________|
AG +  | 2307   | 1868    | 4175
AG -  | 2174   | 126178  | 128352 
      | 4481   | 128046

Total: 132527
 Sand positiv (AG+,PCR+): 51.48 % af alle PCR+
Falsk negativ (AG-,PCR+): 48.52 % af alle PCR+
 Sand positiv AG (AG+,PCR+): 55.26 % af alle AG+
Falsk positiv AG (AG+,PCR-): 44.74 % af alle AG+


In [203]:
da = df2.Dato
TP = df2.AGpos_PCRpos
FP = df2.AGposPCRneg
FN = df2.AGnegPCRpos
TN = df2.AGnegPCRneg

N = TP+FP+FN+TN

DOR = (TP*TN)/(FP*FN)

PPV = TP/(TP+FP)
FDR = FP/(TP+FP)
FOR = FN/(TN+FN)
NPV = TN/(TN+FN)

TPR = TP/(TP+FN)
FNR = FN/(TP+FN)
FPR = FP/(TN+FP)
TNR = TN/(TN+FP)

PRE = (TP+FN)/N
ACC = (TP+TN)/N

LRP = TPR/FPR
LRN = FNR/TNR


DOR2 = LRP/LRN


F1 = 2 * (PPV*TPR)/(PPV+TPR)

# fig,ax1 = plt.subplots(1,1)
# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
# fig,(ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,sharex=True)

ax1.plot(da,PPV,label='Positive predictive value (Precision)')
ax1.plot(da,FDR,label='False discovery rate')
ax1.plot(da,FOR,label='False omission rate')
ax1.plot(da,NPV,label='Negative predictive value')


ax2.plot(da,TPR,label='True positive rate')
ax2.plot(da,FNR,label='False negative rate')
ax2.plot(da,FPR,label='False positive rate')
ax2.plot(da,TNR,label='True negative rate (Specificity)')


ax3.plot(da,PRE,label='Prevalence')
ax3.plot(da,ACC,label='Accuracy')

ax4.plot(da,LRP,label='Positive likelihood ratio')
ax4.plot(da,LRN,label='Negative likelihood ratio')

# plt.plot(da,TP)
# plt.plot(da,FP)
# plt.plot(da,FN)
# plt.plot(da,TN)

ax4.plot(da,DOR,label='Diagnostic odds ratio')

ax4.plot(da,F1,label='F1 score')
ax4.set_ylim(bottom=0)

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax4.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [214]:
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4,1,sharex=True)

ax1.plot(da,PPV,label='Positive predictive value (Precision)')
ax1.plot(da,TPR,label='True positive rate')
ax1.plot(da,F1,label='F1 score')
# ax1.plot(da,FDR,label='False discovery rate')
# ax1.plot(da,FOR,label='False omission rate')
# ax1.plot(da,NPV,label='Negative predictive value')


ax2.plot(da,TNR,label='True negative rate (Specificity)')
ax2.plot(da,NPV,label='Negative predictive value')
ax2.plot(da,ACC,label='Accuracy')

ax3.plot(da,FPR,label='False positive rate')
ax3.plot(da,FOR,label='False omission rate')
ax3.plot(da,PRE,label='Prevalence')

# ax2.plot(da,TPR,label='True positive rate')
# ax2.plot(da,FNR,label='False negative rate')
# ax2.plot(da,FPR,label='False positive rate')
# ax2.plot(da,TNR,label='True negative rate (Specificity)')


# ax3.plot(da,PRE,label='Prevalence')
# ax3.plot(da,ACC,label='Accuracy')

ax4.plot(da,LRP,label='Positive likelihood ratio')
# ax4.plot(da,LRN,label='Negative likelihood ratio')

# # plt.plot(da,TP)
# # plt.plot(da,FP)
# # plt.plot(da,FN)
# # plt.plot(da,TN)

ax4.plot(da,DOR,label='Diagnostic odds ratio')

ax4.set_ylim(bottom=0,top=300)

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax3.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax4.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [249]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.plot(da,N,label='Antal tests')
# ax1.plot(df2.Dato,df2.AG_testede,label='Antal tests')

ax2.plot(da,DOR,label='Diagnostic odds ratio')


meanWidth = 7
rmDa = rnTime(da,meanWidth)
rmDOR = rnMean(DOR,meanWidth)
ax2.plot(rmDa,rmDOR,'k--',label=str(meanWidth)+' dages løbende gennemsnit')

ax1.plot(rmDa,rnMean(N,meanWidth),'k--',label=str(meanWidth)+' dages løbende gennemsnit')

ax1.set_ylim(bottom=0)
ax2.set_ylim([0,250])
ax2.set_xlim([da.iloc[0],da.iloc[-1]])

ax1.grid()
ax2.grid()

ax1.legend()
ax2.legend()

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b\n%Y'))
plt.tight_layout()

KeyboardInterrupt: 

In [42]:
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.plot(df2.Dato,df2['Andel_AGpos_PCRkonf_pct'])
ax2.plot(df2.Dato,df2['Andel_AGminusPCRkonf_pct'])

lastDate = df2.iloc[-1].Dato
# ax1.set_xlim([np.datetime64('2021-02-01'),today])
ax1.set_xlim([np.datetime64('2021-02-01'),lastDate])

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b\n%Y'))
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [189]:
# df2

In [21]:
# Test of various stuff

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)


# ax1.plot(df.Date,df.Tested)
ax1.plot(df2.Dato,df2['AGnegPCRpos'],'k',label='AG neg, PCR pos')

# ax1.plot(df2.Dato,df2.AG_testede,'k',label='AG neg, PCR pos')

ax2.plot(df2.Dato,100*np.divide(df2['AGnegPCRpos'],df2['AG_testede']),'k',label='AG neg, PCR pos')
# ax1.plot(df2.Dato,df2['AG_testede'],'--')
# ax1.plot(df.Date,df.Tested,':')

# curYlim = ax1.get_ylim()


# sens = 0.99 # TPR
# spec = 0.5 # TNR
# fnr = 1 - sens 
# fpr = 1 - spec
# ax1.plot(df.Date,fnr*df.Tested,label='Forventet antal falske negative')

sensToShow = [0.99,0.995,0.998,0.999]
sensToShow = [0.9975,0.998,0.9985,0.999,0.9995]

for sens in sensToShow:
    fnr = 1 - sens
    # ax1.plot(df.Date,fnr*df.Tested,'--',label='Forventet antal falske negative, sensitivitet: '+str(sens))
    # sensLabel = str(round(100*sens,2))
    sensLabel = "{:2.2f}".format(100*sens)
    # ax1.fill_between(df.Date,fnr*df.Tested,label='Forventet ved sensitivitet: '+sensLabel + ' %')
    ax1.fill_between(df2.Dato,df2['AG_testede']*fnr,label='Forventet ved sensitivitet: '+sensLabel + ' %')

    ax2.fill_between(df.Date,100*fnr*np.ones(df.Date.shape),label='Forventet ved sensitivitet: '+sensLabel+ ' %')

    
# ax2.plot(df2.Dato,100*np.divide(df2['AGnegPCRpos'],df2['AG_testede']),'k',label='__nolegend__')
# ax1.plot(df2.Dato,df2['AGnegPCRpos'],'k',label='__nolegend__')

# ax1.set_ylim([0,curYlim[1]])
ax1.set_ylim(bottom=0)
ax2.set_ylim([0,100*(1-sensToShow[0])])

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

ax1.set_ylabel('Antal falsk negative tests')
ax2.set_ylabel('Andel falsk negative tests [%]')

lastDate = df.iloc[-1].Date
ax1.set_xlim([np.datetime64('2021-02-01'),lastDate])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b\n%Y'))
plt.tight_layout()


if saveFigures:
    fig.savefig('figs/FalskNegativeAG')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
# Test of various stuff

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)

ax1.plot(df2.Dato,df2['AGposPCRneg'],'k',label='AG pos, PCR neg')

ax2.plot(df2.Dato,100*np.divide(df2['AGposPCRneg'],df2['AG_testede']),'k',label='AG pos, PCR neg')

# specsToShow = [0.99,0.995,0.998,0.999]
# specsToShow = [0.99,0.995,0.998,0.999]
# specsToShow = [0.9975,0.998,0.9985,0.999,0.9995]
specsToShow = [0.997,0.9975,0.998,0.9985,0.999,0.9995]

for spec in specsToShow:
    fpr = 1 - spec
    specLabel = "{:2.2f}".format(100*spec)
    # ax1.fill_between(df.Date,fpr*df.Tested,label='Forventet ved specificitet: '+specLabel + ' %')
    ax1.fill_between(df2.Dato,df2['AG_testede']*fpr,label='Forventet ved specificitet: '+specLabel + ' %')

    ax2.fill_between(df.Date,100*fpr*np.ones(df.Date.shape),label='Forventet ved specificitet: '+specLabel + ' %')
    
# ax2.plot(df2.Dato,100*np.divide(df2['AGnegPCRpos'],df2['AG_testede']),'k',label='__nolegend__')
# ax1.plot(df2.Dato,df2['AGnegPCRpos'],'k',label='__nolegend__')

# ax1.set_ylim([0,curYlim[1]])
ax1.set_ylim(bottom=0)
ax2.set_ylim([0,100*(1-specsToShow[0])])

ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

ax1.set_ylabel('Antal falsk positive tests')
ax2.set_ylabel('Andel falsk positive tests [%]')

lastDate = df.iloc[-1].Date
ax1.set_xlim([np.datetime64('2021-02-01'),lastDate])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b\n%Y'))
plt.tight_layout()
if saveFigures:
    fig.savefig('figs/FalskPositiveAG')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:

# fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig,ax1 = plt.subplots(1,1)

# ax1.plot(df.Date,df.Tested,'.')
# ax1.plot(dfPCR.Date,dfPCR.Tested,'.')

ax1.plot(dfPCR.Date,dfPCR.NewPositive * 1e5 / 5840000,'b.')
beta = 0.55
ax1.plot(dfPCR.Date,dfPCR.NewPositive * ((1e5/dfPCR.Tested)**beta) * 1e5 / 5840000,'r.')


lastDate = df.iloc[-1].Date
ax1.set_xlim([np.datetime64('2021-01-10'),lastDate])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b\n%Y'))
ax1.set_ylim(bottom=0,top = 50)
ax1.grid()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
firstDate = np.datetime64('2021-02-10')
# lastDate = today
# lastDate = df2.iloc[-1,0]
lastDate = np.datetime64(df2.iloc[-1,0]).astype('datetime64[D]')
curInterval = np.arange(firstDate,lastDate)

indexPCR = dfPCR.Date.isin(curInterval)
indexAG  = df.Date.isin(curInterval)
indexAG2 = df2.Dato.isin(curInterval)

datePCR = dfPCR.Date[indexPCR]
dateAG = df.Date[indexAG]
dateAG2 = df2.Dato[indexAG2]

testPCR = dfPCR.Tested[indexPCR].values
testAG = df.Tested[indexAG].values
testAG2 = df2.AG_testede[indexAG2].values


posPCR = dfPCR.NewPositive[indexPCR].values
posAG = df.NewPositive[indexAG].values
posAG2 = df2.AG_pos[indexAG2].values

posOnlyPCR = posPCR - posAG2
testOnlyPCR = testPCR - posAG2 # Remove those who were only tested because of a positive AG test

beta = 0.55
inciPCR = posPCR * 1e5 / 5_840_000
inciPCRNorm = inciPCR * ((1e5/testPCR)**beta) 
# inciPCRNorm = inciPCR * ((1e5/dfPCR.Tested[indexPCR])**beta) 

inciOnlyPCR = posOnlyPCR * 1e5 / 5_840_000
inciOnlyPCRNorm = inciOnlyPCR * ((1e5/testOnlyPCR)**beta) 


posAGpos = df2.AGpos_PCRpos[indexAG2].values

fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)


# ax1.plot(datePCR,5_840_000*inciPCR/1e5,'b.')
# ax1.plot(datePCR,5_840_000*inciPCRNorm/1e5,'r.')
ax1.plot(datePCR,inciPCR,'b.')
ax1.plot(datePCR,inciPCRNorm,'r.')
# ax1.plot(datePCR,inciOnlyPCR,'b*')
# ax1.plot(datePCR,inciOnlyPCRNorm,'r*')

# ax2.plot(dateAG,posAG)
# ax2.plot(dateAG2,posAG2)
# ax2.plot(datePCR,dfPCR.NewPositive[indexPCR])

# ax2.plot(dateAG,np.divide(testAG,inciPCR),'b.')
ax2.plot(dateAG,np.divide(np.multiply(testAG,inciPCR),1e5),'b.')
# ax2.plot(dateAG,np.divide(np.multiply(testAG,inciOnlyPCR),1e5),'b*')
# ax2.plot(dateAG,np.divide(np.multiply(testAG,inciPCRNorm),1e5),'r.')
ax2.plot(dateAG2,posAGpos,'.')


ax1.set_xlim([firstDate,lastDate])
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d-%b\n%Y'))
ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)

ax1.grid()
ax2.grid()
plt.tight_layout()

# plt.figure()
# # plt.plot(df.Date,df.Tested,'.')
# # plt.plot(df2.Dato,df2.AG_testede,'.')
# plt.plot(df2.Dato,100 * np.divide(df2.AG_pos,df2.AG_testede),'.',label='AG positiv procent')
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
# len(testAG)
# len(inciPCR) 

# plt.figure()
# plt.plot(dateAG2,testAG2*(10/1e5),'.')
# plt.show()
# lastDate = np.datetime64(df2.iloc[-1,0]).astype('datetime64[D]')